# Understanding Session

In [1]:
from IPython.display import Markdown
import lionagi as li

The `Session` object is at the core of lionagi.

A session is an interface to
- manages and logs various messages (system, user, assistant) in a conversation, 
- call API with rate_limit, and 
- enable effortless multi-round exchange between many participants. 

A Session object contains a `Conversation` object, which contains an `Message` object. 
- `Conversation`: create, manage, keep track of messages in a single conversation
- `Message`: The smallest unit of data with a role


**Note:**
currently Session can only call ChatCompletion, support for other OpenAI API endpoints are under developments. currently only supports one assistant in a conversation, multi assistant will be supported in future

## 1. Initiate and Followup

Let us start with a simple use case of the `Session` object, which is to have a multi-turn conversation with an intelligence provider, i.e. OpenAI API service. 

In [2]:
# create some messages
sys_comedian = "As a comedian, you are sarcastically funny"
instruct1 = "very short joke: a blue whale and a big shark meet at the bar and start dancing"
instruct2 = "continue the short joke: the blue whale was confused why it was dancing with a shark"

In [3]:
# create a session with the comedian and generate some jokes in a multi-turn conversation
# also input target output directory for saving the conversation logs
comedian = li.Session(system=sys_comedian, dir="data/logs/comedian/")

In [5]:
joke1 = await comedian.initiate(instruct1)
joke2 = await comedian.followup(instruct2)

# you can continue the conversation with more messages until out of context length
# joke3 = await comedian.followup(instruct3)
# joke4 = await comedian.followup(instruct4)

In [6]:
Markdown(joke1)

Why did the blue whale and the big shark get kicked out of the bar for dancing? Because every time they hit the dance floor, they turned it into a sea-quence nobody could survive! 🐋💃🦈🕺

In [7]:
Markdown(joke2)

The blue whale says to the shark, "I'm confused, I thought we were supposed to be enemies!" The shark grins and replies, "Don't worry, we're just taking a bite out of the dance floor, not each other!"

## 2. Using a second Session as Validation

In [8]:
# some messages from the critic
sys_critic = "you are a respected commentator, you are artistically logical"
instruct3 = "short comments, what do you think about the first joke?"
instruct4 = "provide a concise artistic critique on both jokes, and rate from 1-10"

In [9]:
# create a session with the critic and generate some comments in a multi-turn conversation
critic = li.Session(sys_critic, dir="data/logs/critic/")

# provide context for the critic
context = {
    "joke1": {"prompt": instruct1, "response": joke1},
    "joke2": {"prompt": instruct2, "response": joke2}
}

# you only need to provide same context once in a conversation
comment1 = await critic.initiate(instruct3, context=context)    
comment2 = await critic.followup(instruct4)

In [10]:
Markdown(comment1)

The first joke plays delightfully with expectations, blending the grandeur of ocean giants with the light-heartedness of a dance-off. It's a whimsical scenario that's sure to provoke a smile with its clever punchline.

In [11]:
Markdown(comment2)

Joke 1 offers an imaginative twist, presenting a scenario where the enormity of the characters' dance moves becomes a literal sea change. It's a playful use of wordplay and imagery, creating a vivid and humorous picture. The punchline cleverly ties the physicality of the creatures to the aftermath of their actions. Rating: 7/10 for creativity and visual humor.

Joke 2 builds on the theme of unlikely camaraderie, with a punchline that hinges on the surprise element of a natural predator-prey relationship turned into a dance partnership. The humor here is more subtle, relying on the subversion of expectations and the amusing notion of 'biting' the dance floor. Rating: 6/10 for a softer laugh but a smart play on words.

## 3. Using the Validation to improve first session performance

In [12]:
# messages for comedian again
instruct5 = "your jokes were evaluated by a critic, does it make sense to you? why?"
instruct6 = "based on your reflection, write joke1 again"
instruct7 = "write joke2 again"

# provide context for the comedian
context2 = {"comments": critic.conversation.responses}

# run the instructions
reflect = await comedian.followup(instruct5, context=context2)
joke11 = await comedian.followup(instruct6)
joke22 = await comedian.followup(instruct7)

In [13]:
Markdown(reflect)

Absolutely, the evaluations make sense. The critics have highlighted the strengths of the jokes, noting the imaginative scenarios and wordplay that create a visual and humorous picture. They've rated the jokes based on creativity, the use of surprise elements, and the play on words, which are all key components in crafting effective humor. The ratings reflect an appreciation for the subversion of expectations and the whimsical blending of oceanic themes with playful human activities. These critiques provide valuable feedback, indicating that the jokes manage to entertain through their clever punchlines and unexpected twists.

In [14]:
Markdown(joke11)

At a seafloor bar, a blue whale and a big shark start grooving together. The bartender looks on, shakes his head and mutters, "Well, there goes the neighborhood—every time they drop the bass, it's a tsunami!" 🌊🎶🐋🦈

In [15]:
Markdown(joke22)

The blue whale, befuddled mid-twirl, blurts out to the shark, "Why are we dancing? I thought you were all about that 'shark attack' life!" The shark winks and replies, "Relax, I only take a bite on singles night, and right now, I'm just after the 'fin'-ale!" 🦈🕺💃🎶

#### Save the logs

In [16]:
# create seperate log for each session, and save both messages and llm api logs to seperate csv files

comedian.messages_to_csv()
comedian.log_to_csv()

critic.messages_to_csv()
critic.log_to_csv()

11 logs saved to data/logs/comedian/_messages_2023-12-27T09_44_58_208985.csv
5 logs saved to data/logs/comedian/_llmlog_2023-12-27T09_44_58_209592.csv
5 logs saved to data/logs/critic/_messages_2023-12-27T09_44_58_210273.csv
2 logs saved to data/logs/critic/_llmlog_2023-12-27T09_44_58_210676.csv
